In [82]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from utils.classifications_utils import *
from utils.data_processing_utils_lin_edit import *
from utils.data_visualization_utils import *
from utils.metrics_utils import *
from utils.grid_search_utils import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy import signal
from utils.reduce_noise_utils import *
import pickle
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Drone 4 Data

In [83]:
level_0_wind = load_data(0, 6, "drone2")
level_0_wind = separate_data_based_on_apparatus(level_0_wind)

level_1_wind = load_data(1, 6, "drone2")
level_1_wind = separate_data_based_on_apparatus(level_1_wind)

level_2_wind = load_data(2, 6, "drone2")
level_2_wind = separate_data_based_on_apparatus(level_2_wind)

level_3_wind = load_data(3, 6, "drone2")
level_3_wind = separate_data_based_on_apparatus(level_3_wind)

In [84]:
acc_no_wind, gyro_no_wind, _, _ = level_0_wind.values()
label_0 = [0 for x in range(acc_no_wind.shape[0])]

acc_level_1_wind, gyro_level_1_wind, _, _ = level_1_wind.values()
label_1 = [1 for x in range(acc_level_1_wind.shape[0])]

acc_level_2_wind, gyro_level_2_wind, _, _ = level_2_wind.values()
label_2 = [2 for x in range(acc_level_2_wind.shape[0])]

acc_level_3_wind, gyro_level_3_wind, _, _ = level_3_wind.values()
label_3 = [3 for x in range(acc_level_3_wind.shape[0])]

# Reduce noise

In [85]:
gyro_no_wind_reduced = reduce_noise_with_FFT(gyro_no_wind, [100, 300, 200])# Reduce noise

In [86]:
gyro_level_1_reduced = reduce_noise_with_FFT(gyro_level_1_wind, [2500, 5000, 2000])

In [87]:
gyro_level_2_reduced = reduce_noise_with_FFT(gyro_level_2_wind, [6000, 2000, 1000])

In [88]:
gyro_level_3_reduced = reduce_noise_with_FFT(gyro_level_3_wind, [9000, 5000, 2000])

# Split No Wind Data

In [89]:
X_train_acc_0, X_test_acc_0, y_train_acc_0, y_test_acc_0 = \
train_test_split(acc_no_wind, label_0, test_size=0.2, shuffle=False)

X_train_gyro_0, X_test_gyro_0, y_train_gyro_0, y_test_gyro_0 = \
train_test_split(gyro_no_wind_reduced, label_0, test_size=0.2, shuffle=False)

# Split Level 1 Wind

In [90]:
X_train_acc_1, X_test_acc_1, y_train_acc_1, y_test_acc_1 = \
train_test_split(acc_level_1_wind, label_1, test_size=0.2, shuffle=False)

X_train_gyro_1, X_test_gyro_1, y_train_gyro_1, y_test_gyro_1 = \
train_test_split(gyro_level_1_reduced, label_1, test_size=0.2, shuffle=False)

# Split Level 2 Wind

In [91]:
X_train_acc_2, X_test_acc_2, y_train_acc_2, y_test_acc_2 = \
train_test_split(acc_level_2_wind, label_2, test_size=0.2, shuffle=False)

X_train_gyro_2, X_test_gyro_2, y_train_gyro_2, y_test_gyro_2 = \
train_test_split(gyro_level_2_reduced, label_2, test_size=0.2, shuffle=False)

# Split Level 3 Wind

In [92]:
X_train_acc_3, X_test_acc_3, y_train_acc_3, y_test_acc_3 = \
train_test_split(acc_level_3_wind, label_3, test_size=0.2, shuffle=False)

X_train_gyro_3, X_test_gyro_3, y_train_gyro_3, y_test_gyro_3 = \
train_test_split(gyro_level_3_reduced, label_3, test_size=0.2, shuffle=False)

# Stack data up

# Gyro

In [93]:
X_train_gyro = X_train_gyro_0.append(X_train_gyro_1)
X_train_gyro = X_train_gyro.append(X_train_gyro_2)
X_train_gyro = X_train_gyro.append(X_train_gyro_3)
y_train_gyro = np.hstack((y_train_gyro_0, y_train_gyro_1, y_train_gyro_2, y_train_gyro_3))
display(X_train_gyro.shape)

(115200, 3)

In [94]:
X_test_gyro = X_test_gyro_0.append(X_test_gyro_1)
X_test_gyro = X_test_gyro.append(X_test_gyro_2)
X_test_gyro = X_test_gyro.append(X_test_gyro_3)
y_test_gyro = np.hstack((y_test_gyro_0, y_test_gyro_1, y_test_gyro_2, y_test_gyro_3))

# Accelerometer

In [95]:
X_train_acc = X_train_acc_0.append(X_train_acc_1)
X_train_acc = X_train_acc.append(X_train_acc_2)
X_train_acc = X_train_acc.append(X_train_acc_3)
y_train_acc = np.hstack((y_train_acc_0, y_train_acc_1, y_train_acc_2, y_train_acc_3))

In [96]:
X_test_acc = X_test_acc_0.append(X_test_acc_1)
X_test_acc = X_test_acc.append(X_test_acc_2)
X_test_acc = X_test_acc.append(X_test_acc_3)
y_test_acc = np.hstack((y_test_acc_0, y_test_acc_1, y_test_acc_2, y_test_acc_3))

# Transform and generate features

In [97]:
from transformers.featureGenerator import FeatureGenerator

# Train a random forest classifier and evaluate 

In [98]:
from sklearn.ensemble import RandomForestClassifier

# Classify 0, min and max levels of airflows

# Gyro

In [99]:
feature_generator_gyro2 = FeatureGenerator(1)
feature_generator_gyro2.fit(X_train_gyro, 4)

FeatureGenerator(sliding_window=1)

In [100]:
X_train_gyro = feature_generator_gyro2.transform(X_train_gyro, 'gyro')
y_train_gyro = reduce_label_amount(y_train_gyro, 4)

28701


In [101]:
feature_generator_gyro_test2 = FeatureGenerator(1)
feature_generator_gyro_test2.fit(X_test_gyro, 4)

FeatureGenerator(sliding_window=1)

In [102]:
X_test_gyro = feature_generator_gyro_test2.transform(X_test_gyro, 'gyro')
y_test_gyro = reduce_label_amount(y_test_gyro, 4)

7101


# run pickled random forest classifier and predict

In [103]:
from sklearn.externals import joblib

In [109]:
pickled_clf_2 = pickle.load(open("randomforest_gyro_drone1.sav", 'rb'))
y_pred_1 = pickled_clf_2.predict(X_test_gyro)
pickled_clf_2.score(X_test_gyro, y_test_gyro)

0.9119490212646106

In [110]:
get_confusion_matrix(y_test_gyro, y_pred_1)

,actual_0,actual_1,actual_2,actual_3
predicted_0,7084,17,0,0
predicted_1,25,7044,32,0
predicted_2,0,16,7040,45
predicted_3,0,0,2366,4735


In [111]:
# joblib.dump(best_model_rf, FOLDER_PATH + 'random_forest_gyro.pkl')
pickled_clf = joblib.load(FOLDER_PATH + 'random_forest_gyro.pkl')

In [112]:
y_pred = pickled_clf.predict(X_test_gyro)
pickled_clf.score(X_test_gyro, y_test_gyro)

0.5054217715814674

# Confusion Matrix: Gyro

In [113]:
get_confusion_matrix(y_test_gyro, y_pred)

,actual_0,actual_1,actual_2,actual_3
predicted_0,7092,9,0,0
predicted_1,740,6355,6,0
predicted_2,0,6408,690,3
predicted_3,0,941,5941,219
